# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [8]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [9]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [10]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [11]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist - 0
- firstName of user - 1
- gender of user - 2
- item number in session - 3 
- last name of user- 4
- length of the song - 5
- level (paid or free song) - 6
- location of the user - 7
- sessionId - 8
- song title - 9
- userId - 10

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [12]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [13]:
# A Keyspace called sparkify is created
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [14]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [15]:
"""
For Creation of table for the first query, we are looking at the `WHERE condition` which says that the team requires
the artist, song_title, length_of_song based on a session_id and item_in_session.
By looking at glance of the data it can be noted that both session_id and item_in_session are repeated. However, 
session_id has a link with the user_id. So, choose that as the primary key (partition_key) will allow better distribution of the data
amongst multiple nodes done through consistent hashing technique thus making faster fetching of results.

Finally, the chosen primary key is (session_id, item_in_session)

Q1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

"""
query = """CREATE TABLE
           IF NOT EXISTS songs_by_session_items(
           session_id int,
           item_in_session int,
           artist text,
           song_title text,
           song_length float,
           PRIMARY KEY(session_id, item_in_session)
           );
"""
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skips the header
    for line in csvreader:
        query = "INSERT INTO songs_by_session_items(session_id, item_in_session, artist, song_title, song_length) "
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
query = """
SELECT artist, song_title, song_length
FROM songs_by_session_items
WHERE session_id=%s and item_in_session=%s;
"""
try:
    rows = session.execute(query, (338, 4))
except Exception as e:
    print(e)

for row in rows:
    print(f"{row.artist}|{row.song_title}|{row.song_length}")

Faithless|Music Matters (Mark Knight Dub)|495.30731201171875


# Indexing of columns for references
```
- artist - 0
- firstName of user - 1
- gender of user - 2 
- item number in session - 3 
- last name of user- 4
- length of the song - 5 
- level (paid or free song) - 6 
- location of the user - 7 
- sessionId - 8 
- song title - 9 
- userId - 10 
```

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [18]:
"""
For Creation of table for the second query, we are looking at the `WHERE condition` which says that the team requires
the name of artist, song (sorted by itemInSession) and user (first and last name) based on a userid = 10, sessionid = 182.
By looking at the glance of the data above, it is can be noticed that session_id is linked with user_id. Therefore, 
it is better to make both (user_id, session_id) as a composite partition key to not only allow uniqueness but also 
better placement of data amongst multiple nodes. One thing that will be important is to maintain 
the same order as the created composite_key in the WHERE clause and then the clustering_columns.

Since, the query asks us to sort the data by item_in_session that is included in the clustering_columns as they 
help in sorting the data in ascending order (alphabetical order). I also test the query with `ORDER BY item_in_session;` 
this as well works but is not necessary.

Finally, the choosen PRIMARY KEY is as ((user_id, session_id), item_in_session)

Q2: Give me only the following: name of artist, 
song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
"""
query = """CREATE TABLE
           IF NOT EXISTS songs_by_user_session(
           user_id int,
           session_id int,
           item_in_session int,
           first_name text,
           last_name text,
           artist text,
           song_title text,
           PRIMARY KEY((user_id, session_id), item_in_session)
           );
"""
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skips the header
    for line in csvreader:
        query = "INSERT INTO songs_by_user_session(user_id, session_id, item_in_session, first_name, last_name, artist, song_title) "
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

In [20]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
SELECT artist, song_title, first_name, last_name
FROM songs_by_user_session
WHERE user_id=%s and session_id=%s;
"""
try:
    rows = session.execute(query, (10, 182))
except Exception as e:
    print(e)

for row in rows:
    print(f"{row.artist}||{row.song_title}||{row.first_name}||{row.last_name}")

Down To The Bone||Keep On Keepin' On||Sylvie||Cruz
Three Drives||Greece 2000||Sylvie||Cruz
Sebastien Tellier||Kilometer||Sylvie||Cruz
Lonnie Gordon||Catch You Baby (Steve Pitron & Max Sanna Radio Edit)||Sylvie||Cruz


In [21]:
"""
Since, the where condition is based on the song_title to obtain all users who listen to it. The partition key is chosen 
as song_title. The PRIMARY KEY (song_title, user_id).

However, I beleive that this approach could allow overwrite of the data due to the fact that the same song_title could have 
been listen by many users, and the data gets overriden. Hence, another clustering column who avoid this.

I also have tried to create an index based on user_id separately on the table to allow querying only on the user_id. 
But, I have also read some important information in that regard https://docs.datastax.com/en/cql-oss/3.3/cql/cql_using/useWhenIndex.html

Q3: Give me every user name (first and last) in my music app history 
who listened to the song 'All Hands Against His Own'
"""

query = """CREATE TABLE
           IF NOT EXISTS users_by_song_title(
           user_id int,
           first_name text,
           last_name text,
           song_title text,
           PRIMARY KEY (song_title, user_id)
           );
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

#query = """CREATE INDEX ON music_library2(user_id)"""

#try:
#    session.execute(query)
#except Exception as e:
#    print(e)

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skips the header
    for line in csvreader:
        query = "INSERT INTO users_by_song_title(user_id, first_name, last_name, song_title) "
        query = query + "VALUES (%s, %s,%s,%s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

In [23]:
query = """
SELECT first_name, last_name
FROM users_by_song_title
WHERE song_title=%s;
"""
try:
    rows = session.execute(query, ("All Hands Against His Own",))
except Exception as e:
    print(e)

for row in rows:
    print(f"{row.first_name}||{row.last_name}")

Jacqueline||Lynch
Tegan||Levine
Sara||Johnson


### Drop the tables before closing out the sessions

In [24]:
queries = ["DROP TABLE IF EXISTS songs_by_session_items", "DROP TABLE IF EXISTS songs_by_user_session", 
           "DROP TABLE IF EXISTS users_by_song_title"]
try:
    for query in queries:
        session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()